In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%load_ext google.cloud.bigquery

Customer journey:
- Time distribution: from visitors to customers
- Pathways: touchpoint mapping

# 1. Time journey

Session per uuid ?
Session per visitor_id ?

In [50]:
%%bigquery df
# 293468823
WITH clean AS 
(
SELECT 
  DISTINCT full_visitor_id , client_id , organization_uuid , country_id , session_id , session_start_time , session_end_time ,
  session_reached_minimal_created , 
  traffic_source_level_3 , traffic_source_source, traffic_source_medium,
  COUNT(DISTINCT organization_uuid) OVER(PARTITION BY full_visitor_id ) as nr_unique_uuid
FROM `izettle-dp-pipeline-prod.legacy_views.google_analytics_session` 
WHERE full_visitor_id NOT IN (--list of visitor id that has no client id and only one unique (NA) uuid and 
                          SELECT
                              full_visitor_id 
                          FROM (
                                SELECT
                                    DISTINCT full_visitor_id , client_id , organization_uuid  ,
                                    DENSE_RANK() OVER(PARTITION BY full_visitor_id ORDER BY client_id DESC, organization_uuid DESC) as rank_entry 
                                    #optimize the code by group by and array_agg()?
                                FROM `izettle-dp-pipeline-prod.legacy_views.google_analytics_session` 
                                WHERE full_visitor_id IN (SELECT 
                                                            DISTINCT full_visitor_id
                                                          FROM `izettle-dp-pipeline-prod.legacy_views.google_analytics_session` 
                                                          WHERE client_id IS NULL AND organization_uuid IS NULL
                                                          )
                                #ORDER BY full_visitor_id , rank_entry
                                      )
                          WHERE rank_entry = 1
                          AND client_id IS NULL
                          AND organization_uuid IS NULL
                          ) 
), 
ga_processed AS (
--delete row with no uuid and nr_unique_uuid per group of visitor id of that row > 1 
-- and fill NA uuid cells (Expected: rows with NA uuid and nr_unique_uuid =0 left)--
SELECT 
  full_visitor_id , client_id , organization_uuid , 
  IFNULL(organization_uuid, MAX(organization_uuid) OVER(PARTITION BY full_visitor_id)) as fill_uuid ,
  country_id , session_id , session_start_time , session_end_time ,
  session_reached_minimal_created ,
  traffic_source_level_3 , traffic_source_source, traffic_source_medium,
  nr_unique_uuid
FROM clean
WHERE 
  organization_uuid IS NOT NULL
  #OR
  #(organization_uuid IS NULL  
  #AND nr_unique_uuid = 0 AND client_id IS NOT NULL) 
  OR
  (organization_uuid IS NULL AND nr_unique_uuid <= 1)
),
merged_campaign AS(
SELECT
  full_visitor_id , 
  IFNULL(fill_uuid, MAX(campaign_data.organization_uuid) OVER(PARTITION BY client_id)) as fill_uuid_2,
  country_id , session_id , session_start_time , session_end_time ,
  session_reached_minimal_created ,
  traffic_source_level_3, traffic_source_source, traffic_source_medium
FROM ga_processed
LEFT JOIN (SELECT
              DISTINCT google_analytics_id , organization_uuid 
           FROM `izettle-dp-pipeline-prod.legacy_views.user_campaign_data`
           WHERE google_analytics_id IN (
                                         SELECT DISTINCT google_analytics_id
                                         FROM `izettle-dp-pipeline-prod.legacy_views.user_campaign_data`
                                         GROUP BY google_analytics_id 
                                         HAVING COUNT(DISTINCT organization_uuid) = 1
                                         ) 
          ) AS campaign_data
ON ga_processed.client_id = campaign_data.google_analytics_id 
),
merged_oc AS (
--customer journey of merchants until registration tracked by ga cookies--
--only ~ 245,000 merchants have the recorded journey. The rest that we have only the last touch point can be found in the customer_acquisition table--
SELECT
  ft.fill_uuid_2 as organization_uuid , oc.organization_created ,
  ft.full_visitor_id, ft.country_id , ft.session_id , ft.session_start_time , ft.session_end_time ,
  ft.session_reached_minimal_created ,
  ft.traffic_source_level_3,
  traffic_source_source, traffic_source_medium
FROM merged_campaign ft
LEFT JOIN `izettle-dp-pipeline-prod.legacy_views.organization_customer` oc
ON ft.fill_uuid_2 =oc.organization_uuid 
WHERE fill_uuid_2 IS NOT NULL
AND oc.organization_created >= ft.session_start_time 
)
SELECT
  uuid_left.organization_uuid ,
  uuid_left.organization_created ,
  uuid_left.channel_level_3 ,
  merged_oc.full_visitor_id, merged_oc.country_id , merged_oc.session_id , merged_oc.session_start_time , merged_oc.session_end_time ,
  merged_oc.session_reached_minimal_created ,
  merged_oc.traffic_source_level_3,
  merged_oc.traffic_source_source, merged_oc.traffic_source_medium
FROM (
      SELECT 
        cas.organization_uuid,
        cas.channel_level_3 , oc.organization_created 
      FROM `izettle-dp-pipeline-prod.legacy_views.customer_acquisition_source` cas
      JOIN `izettle-dp-pipeline-prod.legacy_views.organization_customer` oc
      ON cas.organization_uuid = oc.organization_uuid 
      AND oc.organization_created >= '2017-09-01'
      ) as uuid_left
LEFT JOIN merged_oc
ON uuid_left.organization_uuid = merged_oc.organization_uuid 


In [52]:
df.head()

,organization_uuid,organization_created,channel_level_3,full_visitor_id,country_id,session_id,session_start_time,session_end_time,session_reached_minimal_created,traffic_source_level_3,traffic_source_source,traffic_source_medium
0,fa071550-ebe0-11e8-83df-40d45c040ea4,2018-11-19 09:53:33.477000+00:00,ps_pay_sdk,2630999179256239427,GB,b')UC\x8c\x0f\x03\xf6z\x94\x9c\xe9o5\xba\xb3\x...,2018-11-12 15:00:15+00:00,2018-11-12 15:00:52+00:00,True,ps_pay_sdk,local_partnership,sdk
1,ca984ad0-aea3-11e7-9c60-1170679cd6c1,2017-10-11 16:46:55.997000+00:00,ps_pay_sdk,8800201112688946195,DE,b'\x18g\x8e\xc3\x8d\x8cm\x106\x88\x9eKT\xe74!\...,2017-10-11 16:46:18+00:00,2017-10-11 16:46:57+00:00,True,ps_pay_sdk,local_partnership,sdk
2,03dff0c0-b0e4-11e8-a76b-3d10d4d81144,2018-09-05 08:16:39.884000+00:00,ps_product_api,7229325147852738764,NO,b'G0\xbbn\x93\xda\xeb\xed\x81\xd0D\xad\xb6D\\/...,2018-09-05 08:16:21+00:00,2018-09-05 08:25:40+00:00,True,ps_product_api,local_partnership,api
3,18c11ba0-a086-11e7-bff1-bdebf499334e,2017-09-23 17:39:05.946000+00:00,ps_product_api,8157977517660383654,ES,b'\x84\xb9\xfew\xef\x18\x83\xbb\x8d\x87\xbaN{\...,2017-09-22 15:19:00+00:00,2017-09-22 15:19:00+00:00,False,ps_product_api,local_partnership,api
4,18c11ba0-a086-11e7-bff1-bdebf499334e,2017-09-23 17:39:05.946000+00:00,ps_product_api,8157977517660383654,ES,b'3Q\x0b\xc2Uh\xdf\xb1\x90\x0c\xc6U\xdc\x99\xb...,2017-09-23 17:38:35+00:00,2017-09-23 17:47:32+00:00,True,ps_product_api,local_partnership,api


In [51]:
df['organization_uuid'].nunique()

1107000

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1692937 entries, 0 to 1692936
Data columns (total 12 columns):
organization_uuid                  object
organization_created               datetime64[ns, UTC]
channel_level_3                    object
full_visitor_id                    object
country_id                         object
session_id                         object
session_start_time                 object
session_end_time                   object
session_reached_minimal_created    object
traffic_source_level_3             object
traffic_source_source              object
traffic_source_medium              object
dtypes: datetime64[ns, UTC](1), object(11)
memory usage: 155.0+ MB


In [6]:
df.to_csv('customer_journey.csv')

In [54]:
test = df[df['session_reached_minimal_created'] == True]

In [55]:
print(test.shape)
test.head()

(652155, 12)


,organization_uuid,organization_created,channel_level_3,full_visitor_id,country_id,session_id,session_start_time,session_end_time,session_reached_minimal_created,traffic_source_level_3,traffic_source_source,traffic_source_medium
0,fa071550-ebe0-11e8-83df-40d45c040ea4,2018-11-19 09:53:33.477000+00:00,ps_pay_sdk,2630999179256239427,GB,b')UC\x8c\x0f\x03\xf6z\x94\x9c\xe9o5\xba\xb3\x...,2018-11-12 15:00:15+00:00,2018-11-12 15:00:52+00:00,True,ps_pay_sdk,local_partnership,sdk
1,ca984ad0-aea3-11e7-9c60-1170679cd6c1,2017-10-11 16:46:55.997000+00:00,ps_pay_sdk,8800201112688946195,DE,b'\x18g\x8e\xc3\x8d\x8cm\x106\x88\x9eKT\xe74!\...,2017-10-11 16:46:18+00:00,2017-10-11 16:46:57+00:00,True,ps_pay_sdk,local_partnership,sdk
2,03dff0c0-b0e4-11e8-a76b-3d10d4d81144,2018-09-05 08:16:39.884000+00:00,ps_product_api,7229325147852738764,NO,b'G0\xbbn\x93\xda\xeb\xed\x81\xd0D\xad\xb6D\\/...,2018-09-05 08:16:21+00:00,2018-09-05 08:25:40+00:00,True,ps_product_api,local_partnership,api
4,18c11ba0-a086-11e7-bff1-bdebf499334e,2017-09-23 17:39:05.946000+00:00,ps_product_api,8157977517660383654,ES,b'3Q\x0b\xc2Uh\xdf\xb1\x90\x0c\xc6U\xdc\x99\xb...,2017-09-23 17:38:35+00:00,2017-09-23 17:47:32+00:00,True,ps_product_api,local_partnership,api
6,58d193a0-db3c-11e7-b3a9-0908ec83f729,2017-12-07 10:49:49.274000+00:00,ps_pay_sdk,4364092837620953880,GB,b'\x926X\x9a\xde+\xa8H\xa6\xee\x16\xfaiy\x8e>\...,2017-12-07 10:49:39+00:00,2017-12-07 11:06:41+00:00,True,ps_pay_sdk,local_partnership,sdk


In [56]:
test['organization_uuid'].nunique()

646122

In [37]:
test[test['organization_uuid'].duplicated()].sort_values('organization_uuid')

,organization_uuid,organization_created,channel_level_3,full_visitor_id,country_id,session_id,session_start_time,session_end_time,session_reached_minimal_created,traffic_source_level_3,traffic_source_source,traffic_source_medium
1390156,01664de8-3b5f-11e9-9afa-8047707a6725,2019-02-28 13:44:44.292000+00:00,o_organic_null,3435330658408753932,FR,b'\xe6o\xe4\xe6Q\xfe\xadr\x01\xc3\xc7\xdd\xf3\...,2019-02-27 11:55:20+00:00,2019-02-27 12:32:33+00:00,True,o_organic_direct,(direct),(none)
1306347,019f7208-15a8-11e9-8894-ccb2c34e0385,2019-01-11 13:51:33.742000+00:00,o_organic_null,4069701287981320567,SE,b'Z4\xe6\x1c\x12O\xb1\xc6\xa1q\xcd\xfbPx\x125\...,2018-12-27 10:50:27+00:00,2018-12-27 11:06:08+00:00,True,o_organic_seo,google,organic
1070513,0309dc8c-610c-11e9-b212-a71ca56ef252,2019-04-17 12:26:22.999000+00:00,o_paid_sem,7244042862629010770,FR,b'c\xa8\xc5\xe0\xd8[\x10\x1br\xe3w\xa9-\xdbJ\x...,2019-04-15 17:20:02+00:00,2019-04-15 17:35:52+00:00,True,o_paid_sem,google,cpc
1362387,03be9c20-93fa-11e7-81a5-5923fd2190bf,2017-09-07 18:26:07.202000+00:00,ds_keyaccount_keyaccount,8414259367262483482,ES,b'\xa7~-A\xc1\x16G)\x8c/\xdc\xb7\xd9\xe4\x92UC...,2017-09-06 07:25:49+00:00,2017-09-06 07:40:19+00:00,True,o_organic_null,crm,email
893849,040bbb40-b858-11e7-837a-06e4cb3c4359,2017-10-24 01:09:42.260000+00:00,o_paid_sem,3781078884355136303,FR,b'\xea\xaa\x88J\x9a\x8c\xeb\xa7\xe8n\xd9\xe5]U...,2017-09-12 13:03:43+00:00,2017-09-12 13:07:17+00:00,True,o_paid_sem,google,cpc
...,...,...,...,...,...,...,...,...,...,...,...,...
1345309,fc60b8e0-ad22-11e7-8970-81dab73a06ec,2017-10-09 18:52:23.278000+00:00,ps_other_retail,8342053352042635916,MX,b'\xb3W\x90(\xbd\xd1#\\/ \x80\xea$\xec\x0cY\x9...,2017-10-09 16:23:40+00:00,2017-10-09 16:24:21+00:00,True,o_paid_social,facebook,comment
941613,fd9076c0-efa6-11e8-b589-a379c1ffc2eb,2018-11-24 05:08:33.251000+00:00,o_paid_sem,1725604877666136933,MX,b'H\x15B\x0c\xa6=\xc2\xdd#\xd0\xef\xe4\xb3\xb7...,2018-11-24 02:11:50+00:00,2018-11-24 02:36:48+00:00,True,o_paid_sem,google,cpc
1324142,fe13d468-30ea-11e9-b31a-eaa575cb316b,2019-02-15 06:29:05.470000+00:00,ds_keyaccount_keyaccount,6319772905615991964,DE,b'v\x1e\x8a\xe3\x10\x05\xd9\xbd`\x10\x14\x1d\x...,2019-02-12 11:55:00+00:00,2019-02-12 12:24:36+00:00,True,o_organic_seo,google,organic
919715,fef3f694-9727-11e9-b268-cfa2b1864b33,2019-06-25 09:02:44.843000+00:00,o_paid_sem,7451082422551671499,GB,b'\x1e`\xc35\x11f\xfd\x99\x05\xd4\xc3\x9d\xac\...,2019-06-06 10:46:33+00:00,2019-06-06 10:57:01+00:00,True,o_paid_sem,google,cpc


In [57]:
duplicated_list = test[test['organization_uuid'].duplicated()]['organization_uuid']

In [58]:
test[test['organization_uuid'].isin(duplicated_list)].sort_values('organization_uuid')

,organization_uuid,organization_created,channel_level_3,full_visitor_id,country_id,session_id,session_start_time,session_end_time,session_reached_minimal_created,traffic_source_level_3,traffic_source_source,traffic_source_medium
1177135,00028e50-8c2b-11e8-86cb-e32affec5342,2018-07-20 14:41:34.645000+00:00,o_organic_seo,7639507628561919997,GB,b'\x96\x97\xfe\xeaL\xdd\x8c\xdc0n]\x05\x8b\xd1...,2018-07-20 14:40:21+00:00,2018-07-20 14:59:45+00:00,True,o_organic_seo,google,organic
1578903,00028e50-8c2b-11e8-86cb-e32affec5342,2018-07-20 14:41:34.645000+00:00,o_organic_seo,7639507628561919997,GB,"b'\xfd\xc2\xcc\x1f\xd71\xd2B\xac\xaaT""\xe0N,\x...",2018-07-20 14:21:50+00:00,2018-07-20 14:24:46+00:00,True,o_paid_affiliate,affiliate,cpc
1110030,001544f0-e032-11e7-9e01-911ab018e404,2017-12-13 18:18:21.247000+00:00,o_organic_seo,7888768413503417984,BR,b'/\x11k\xba\x17\x82\xdd\x80\xda\x10{\xa2J\xe1...,2017-12-13 18:16:32+00:00,2017-12-13 18:31:47+00:00,True,o_organic_seo,google,organic
1267139,001544f0-e032-11e7-9e01-911ab018e404,2017-12-13 18:18:21.247000+00:00,o_organic_seo,423341961373410845,BR,b'\xf5\xa1)\xbd3O4D\xd2b\xce\xe1 \xac\xbf\xb2\...,2017-12-13 16:23:00+00:00,2017-12-13 16:39:45+00:00,True,o_paid_social,facebook,cpc
939497,001c5148-2bc0-11e9-b630-1d7efc924c2e,2019-02-08 16:38:44.708000+00:00,o_paid_sem,3914646530444600992,BR,b'\xbf\x8ape\xa1\xcf{\x03s\x15\x8f|\x91_\xca\x...,2019-02-08 15:56:18+00:00,2019-02-08 16:04:38+00:00,True,o_paid_sem,google,cpc
...,...,...,...,...,...,...,...,...,...,...,...,...
1012067,fff83d00-5904-11e8-b0bc-269b0dd49c83,2018-05-16 12:31:04.400000+00:00,o_paid_sem,7494416550896343521,GB,b'$nNaz1\xb5k\xb9\xfa\xf8\xf42\x905\xc0\xd7+\x...,2018-05-16 12:30:32+00:00,2018-05-16 12:44:01+00:00,True,o_paid_sem,google,cpc
1665276,fffd43bc-6dc2-11e9-8492-749411086a8f,2019-05-03 16:46:29.738000+00:00,ds_keyaccount_keyaccount,5382179152609141965,DK,b'\xb3\x86nr\x98\x0c\xa1\x9eE\xdd\x8e\xd5\x8d\...,2019-04-30 06:01:49+00:00,2019-04-30 06:03:20+00:00,True,ds_keyaccount_keyaccount,local_keyaccount,keyaccount
1665275,fffd43bc-6dc2-11e9-8492-749411086a8f,2019-05-03 16:46:29.738000+00:00,ds_keyaccount_keyaccount,3986321850917374679,DK,b'\xed|o\xf5\x860\x0fyD\x02\xa1\x97f\x87\xbf\x...,2019-05-03 16:46:23+00:00,2019-05-03 17:15:03+00:00,True,ds_keyaccount_keyaccount,local_keyaccount,keyaccount
1482794,fffd60e8-a960-11e9-bb5d-96f4988e377d,2019-07-18 13:36:08.826000+00:00,o_organic_direct,6322915438710714983,GB,b'\xd2\x11\x1a\xd7O\x10\xa2\xb6\xd5-e\x8d\xd2\...,2019-07-18 05:46:36+00:00,2019-07-18 05:49:36+00:00,True,o_organic_direct,(direct),(none)


In [59]:
test[test['organization_uuid'].isin(duplicated_list)]['organization_uuid'].nunique()

5769

In [60]:
df[df['session_id'].isna()]['organization_uuid'].nunique()

422121

In [61]:
df[~df['session_id'].isna()]['organization_uuid'].nunique()

684879

In [48]:
test['organization_uuid'].nunique()

8214